# World Happiness Report EDA

* Dünya Mutluluk Raporu, küresel mutluluk durumuna ilişkin dönüm noktası niteliğindeki bir araştırmadır. İlk rapor 2012'de, ikincisi 2013'te, üçüncüsü 2015'te ve dördüncüsü 2016 Güncellemesinde yayınlandı. 155 ülkeyi mutluluk seviyelerine göre sıralayan World Happiness 2017, 20 Mart Dünya Mutluluk Günü'nü kutlayan bir etkinlikte Birleşmiş Milletler'de yayınlandı. Rapor, hükümetler, kuruluşlar ve sivil toplum politika belirleme kararlarını bilgilendirmek için mutluluk göstergelerini giderek daha fazla kullandıkça, küresel kabul görmeye devam ediyor. Ekonomi, psikoloji, anket analizi, ulusal istatistikler, sağlık, kamu politikası ve daha fazlası gibi alanlarda önde gelen uzmanlar, ülkelerin ilerlemesini değerlendirmek için refah ölçümlerinin nasıl etkili bir şekilde kullanılabileceğini açıklar. Raporlar, bugün dünyadaki mutluluk durumunu gözden geçiriyor ve yeni mutluluk biliminin mutluluktaki kişisel ve ulusal farklılıkları nasıl açıkladığını gösteriyor.

# Columns 
#### Country = Name of the country.
#### Region = Region the country belongs to.
#### Happiness Rank = Rank of the country based on the Happiness Score.
#### Happiness Score = A metric measured in 2015 by asking the sampled people the question: "How would you rate your happiness on a scale of 0 to 10 where 10 is the happiest.
#### Standard Error = The standard error of the happiness score.
#### Economy (GDP per Capita) = The extent to which GDP contributes to the calculation of the Happiness Score
#### Family = The extent to which Family contributes to the calculation of the Happiness Score
#### Health (Life Expectancy) = The extent to which Life expectancy contributed to the calculation of the Happiness Score
#### Freedom = The extent to which Freedom contributed to the calculation of the Happiness Score
#### Trust (Government Corruption) = The extent to which Perception of Corruption contributes to Happiness Score
#### Generosity = The extent to which Generosity contributed to the calculation of the Happiness Score.
#### Dystopia Residual = The extent to which Dystopia Residual contributed to the calculation of the Happiness Score.

* Kütüphanelerimizi yükleyelim

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from IPython.display import Image
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

* pep 8 formatında yazabilmek adına komutlarımızı girelim

* Verilerimizi yükleyelim,okumaya başlayalım

In [ ]:
happiness = pd.read_csv("../input/world-happiness/2015.csv")
happiness.head()

* Herhangi bir aksilik olabilme duruma karşı,orjinal df'i bozmamak adına df'in kopyasını alalım

In [ ]:
df = happiness.copy()

# 1. Veri Seti Hikayesi ve Yapısının İncelenmesi

In [ ]:
df.head()

* df içerisinde kaç gözlem ve özellik var inceleyelim

In [ ]:
df.shape

* df içerisinde 158 gözlem verisi bulunurken, 12 tane özellik bilgisi bulunmaktadır.df info ile içerisindeki özelliklerin yapılarını ve hangi tür veri barındırdıklarını inceleyelim

In [ ]:
df.info()

* Veri kümemiz içerisinde Country,Region obje tipinde değer tutarken, Happiness Rank,Score,Standard Error ve bunun gibi diğer değerler numerik bilgiler içeriyor.Verilerin özellikler içerisinde istatistiksel olarak dağılımlarını inceleyelim

In [ ]:
df.describe().T

* msno matrix ile içerisinde boş değer varmı, yok mu ve bunların diğer kolonlarla olan ilşkisini inceleyelim

In [ ]:
msno.matrix(df)

* Matris içerisinde herhangi bir kolonun herhangi bir satırında bile boş değer bulunmamaktadır.İçeride belki outlier oluşturan aykırı değerler söz konusu olabilir.İlerleyen önişleme adımlarında bunları kullanabiliriz

* Mutluluk datası içerisinde boş değerlerin olup olmadığından emin olalım

In [ ]:
df.isnull().sum()

* Önceden de incelendiği gibi df içerisinde boş veri bulunmamaktadır

In [ ]:
f, ax = plt.subplots(figsize=[20, 15])
sns.heatmap(df.corr(), annot=True, ax=ax, cmap="Blues")

* Elimizde olan df içinde öznitelikler arasında;
**78% Happiness Score - Economy arasında pozitif korelasyon,**
**74% Happiness Score - Family arasında pozitif korelasyon,**
**72% Happiness Score - Health arasında pozitif korelasyon,**
**57% Happiness Score - Freedom arasında pozitif korelasyon,**
**40% Happiness Score - Trust arasında pozitif korelasyon,**
**53% Happiness Score - Distopia Residual arasında pozitif korelasyon,**
**65% Economy - Family arasında pozitif korelasyon,**
**82% Economy - Health arasında pozitif korealsyon,**
**53% Family - Health arasında pozitif korelasyon,**
**37% Freedom - Economy arasında pozitif korelasyon,**
**31% Trust - Economy arasıda pzoitif korelasyon,**
**44% Freedom - Family arasında pozitif korealsyon,**
**49% Trust - Freedom arasında pozitif korealsyon,**
**37% Generosity - Freedom arasında pozitif korelasyon,**
**28% Generosity - Trust arasında pozitif korelasyon** bulunmaktadır.

# 2. Veri Seti İçerisindeki Öznitelikleri İnceleyelim

In [ ]:
df.head()

In [ ]:
visualized_graph = df.hist(figsize=(20, 20))
visualized_graph

## 2.1 Country

In [ ]:
df[["Country"]].head()

In [ ]:
df[["Country"]].describe().T

In [ ]:
df["Country"].unique()

* istatistiksel değerlerini ölçümledikten sonra bu kolon içerisinde bulunan değerlerin sıklıklarını kontrol edelim

In [ ]:
df["Country"].value_counts()

In [ ]:
df.shape

In [ ]:
df.groupby(["Country"]).count()

* 158 farklı kayıtımız ve sıklıkları 1 olduğu için df'in büyüklüğü de 158,12 olduğundan her bir ülke df içerisinde sadece 1 kez geçmiş demek oluyor

## 2.2 Region

* Region içerisinde bulunan gözlemlerimize bakalım

In [ ]:
df[["Region"]]

* En fazla geçen bölgeyi bulalım

In [ ]:
df[["Region"]].mode()

In [ ]:
df[["Region"]].value_counts()

Burada en fazla geçen bölge "Sub-Saharan Africa" verisi 40 tane değer içeriyor, en az "Australia ve New Zealand" ile birlikte "North America" 2 kere geçmiş 

In [ ]:
df["Region"].nunique()

* Region özelliği içerisinde eşsiz değerlerimiz tam olarak 10 tanedir,bu eşsiz değerlerimizi unique() ile görelim

In [ ]:
df["Region"].unique()

## 2.3. Happiness Rank

* Mutluluk değerlerini içeren özelliği inceleyelim

In [ ]:
df["Happiness Rank"]

* Minimum ve maksimum değerleri görüntüleyelim

In [ ]:
df[["Happiness Rank"]].min()

In [ ]:
df[["Happiness Rank"]].max()

In [ ]:
df[["Happiness Rank"]].value_counts()

* Sadece happiness rank içerisinde 82 değeri 2 kere geçmiş.Diğerlerinin hepsi 1 kez kullanılmış.82 olarak görünen gözlemleri inceleyelim

In [ ]:
df[df["Happiness Rank"] == 82]

## 2.4. Happiness Score

In [ ]:
df[["Happiness Score"]].head()

* Mutluluk skorunun minimum ve maximum olduğu değerleri inceleyelim ve sıklıklarını kontrol edelim.

In [ ]:
df[["Happiness Score"]].max()

In [ ]:
df[["Happiness Score"]].min()

In [ ]:
df[["Happiness Score"]].value_counts()

* Mutluluk skorunun 5.192 olarak seyrettiği 2 tane gözlem görülüyor.Bunların neler olduğunu inceleyelim

In [ ]:
df[df["Happiness Score"] == 5.192]

* Happiness Rank ve Score arasında çok yakın bir ilşki olmalı çünkü aynı 2 gözlem değerinde sıklık değerleri kesişiyor

## 2.5. Standard Error

In [ ]:
df["Standard Error"].head()

* df içerisinde minimum ve maksimum neler bulunuyor,bunları inceleyelim.Sıklıklarını kontrol edelim

In [ ]:
df["Standard Error"].min()

In [ ]:
df["Standard Error"].max()

In [ ]:
df["Standard Error"].value_counts()

## 2.6. Economy (GDP per Capita)

In [ ]:
df["Economy (GDP per Capita)"].head()

* Ekonomi kolonunun içerisinde minimum ve maksimum değerşeri kontrol edelim.Sıklıklarını inceleyelim

In [ ]:
df["Economy (GDP per Capita)"].min()

In [ ]:
df["Economy (GDP per Capita)"].max()

In [ ]:
df["Economy (GDP per Capita)"].value_counts()

In [ ]:
df["Economy (GDP per Capita)"].unique()

In [ ]:
df["Economy (GDP per Capita)"].nunique()

* df içerisinde minimum 0,maksimum 1.69 gibi değer meevcutken sıklıklarını görmek yine mümkün değil.Her kayıttan yine df içerisinde 1'er tane mevcut

## 2.7. Family

In [ ]:
df["Family"].head()

* Family kolonu içerisinde minimum,maksimum ve sıklıklarını kontrol edelim

In [ ]:
df["Family"].min()

In [ ]:
df["Family"].max()

In [ ]:
df["Family"].value_counts()

* Sıklık değerleri tek,yani df gözlemi kadar eşsi,z değerimiz var

## 2.8. Health (Life Expectancy)

In [ ]:
df["Health (Life Expectancy)"].head()

In [ ]:
df["Health (Life Expectancy)"].min()

In [ ]:
df["Health (Life Expectancy)"].max()

In [ ]:
df["Health (Life Expectancy)"].value_counts()

df["Health (Life Expectancy)"] içerisinde toplam 157 unique değer var.Bir tane sıklığı 2 olarak görünen 0.92356 değeri var.Bunu inceleyelim

In [ ]:
df.head()

## 2.9. Freedom

* Mutlulukla ilşkili özgürlük oranı olarak tanımlanıyor.Bunun içinde minimum,maksimum ve sıklık değerlerini inceleyelim

In [ ]:
df[["Freedom"]].head()

In [ ]:
df["Freedom"].min()

In [ ]:
df["Freedom"].max()

In [ ]:
df["Freedom"].value_counts()

## 2.10. Trust (Government Corruption)

In [ ]:
df["Trust (Government Corruption)"].head()

In [ ]:
df["Trust (Government Corruption)"].min()

In [ ]:
df["Trust (Government Corruption)"].max()

In [ ]:
df["Trust (Government Corruption)"].value_counts()

Sıklık değeri 1 noktada 2 olarak görünmüş.Buraya vuran değerleri inceleyelim

* Sıklık değeri 2 olarak görülmesine rağmen,gözlemleri göremiyoruz

In [ ]:
df[df["Trust (Government Corruption)"] == 0.32524]

## 2.11. Generosity

In [ ]:
df["Generosity"].head()

* df içerisinde minimum,maksimum ve sıklık değerlerini görüntüleyelim

In [ ]:
df["Generosity"].min()

In [ ]:
df["Generosity"].max()

In [ ]:
df["Generosity"].value_counts()

## 2.12. Dystopia Residual

In [ ]:
df["Dystopia Residual"].head()

* df içerisinde bulunan değerlerin minimum,maksimum ve sıklık değerlerini kontrol edelim

In [ ]:
df["Generosity"].min()

In [ ]:
df["Generosity"].max()

In [ ]:
df["Generosity"].value_counts()

# 3. Veri Önişleme

In [ ]:
df.head()

* Veriler arasında doğrudan bir kalibre edilen bilgiler bulunmadığı için her bir gözlem değeri eşsiz olduğu için aykırı değer var mı,yok mu kontrol edilebilinir

* Burada doğrudan bir hedef bulunmamaktadır.Kümeleme yöntemi ile çözümlenebilir.Yakın amaç insanların mutlu olması olduğu için happiness score değerini alıp,pairplot çizdirelim

# 4. Aykırı Değer Analizi

In [ ]:
fig, [[ax1, ax2, ax3, ax4, ax5], [ax6 ,ax7, ax8, ax9, ax10]] = plt.subplots(nrows=2,
                                      ncols=5, figsize=(14, 20))

ax1.boxplot(df["Happiness Rank"])
ax1.set_title("Happiness Rank")
ax2.boxplot(df["Happiness Score"])
ax2.set_title("Happiness Score")
ax3.boxplot(df["Standard Error"])
ax3.set_title("Standard Error")
ax4.boxplot(df["Economy (GDP per Capita)"])
ax4.set_title("Economy (GDP per Capita)")
ax5.boxplot(df["Family"])
ax5.set_title("Family")
ax6.boxplot(df["Health (Life Expectancy)"])
ax6.set_title("Health (Life Expectancy)")
ax7.boxplot(df["Freedom"])
ax7.set_title("Freedom")
ax8.boxplot(df["Trust (Government Corruption)"])
ax8.set_title("Trust (Government Corruption)")
ax9.boxplot(df["Generosity"])
ax9.set_title("Generosity")
ax10.boxplot(df["Dystopia Residual"])
ax10.set_title("Dystopia Residual")

* Aykırı değerler genellikle Standard Error, Family, Trust ve Distopia Residual kolonları içerisinde gözleniyor

In [ ]:
def detect_outlier(data):
    q1 = np.quantile(data, 0.25)
    q3 = np.quantile(data, 0.75)
    IQR = q3 - q1
    low = q1 - (2 * IQR)
    high = q3 + (2 * IQR)
    return low, high

## 4.1. Standard Error 

In [ ]:
low, high = detect_outlier(df["Standard Error"])

* Sınırlarımızı tanımlayalım ve bu aralıklardaki verilerimizi gözlemleyelim,grafiklerini çizdirelim

In [ ]:
low

In [ ]:
high

In [ ]:
df[df["Standard Error"] > low]

In [ ]:
df[df["Standard Error"] < high]

In [ ]:
df[(df["Standard Error"] > low) & (df["Standard Error"] < high)]

In [ ]:
df  = df[(df['Standard Error'] > low) & (df['Standard Error'] < high)]
df['Standard Error'].plot(kind='box')
plt.figure()
df['Standard Error'].plot(kind='hist', bins=50)

## 4.2. Family

In [ ]:
lowf, highf = detect_outlier(df["Family"])

In [ ]:
lowf

In [ ]:
highf

In [ ]:
df[df["Family"] > lowf]

In [ ]:
df[(df["Family"] > lowf) & (df["Family"] < highf)]

In [ ]:
df  = df[(df['Family'] > lowf) & (df['Family'] < highf)]
df['Family'].plot(kind='box')
plt.figure()
df['Family'].plot(kind='hist', bins=50)

In [ ]:
df.head()

## 4.3. Trust (Government Corruption)

In [ ]:
lowt, hight = detect_outlier(df["Trust (Government Corruption)"])

* Düşük ve yüksek değerler neler.Bu arada kalan değerlerimizi kontrol edip,df'i güncelleyelim

In [ ]:
lowt

In [ ]:
hight

In [ ]:
df[df["Trust (Government Corruption)"] > lowt]

In [ ]:
df[df["Trust (Government Corruption)"] < hight]

In [ ]:
df[(df["Trust (Government Corruption)"] > lowt) & (df["Trust (Government Corruption)"] < hight)]

In [ ]:
df  = df[(df['Trust (Government Corruption)'] > lowt) & (df['Trust (Government Corruption)'] < hight)]
df['Trust (Government Corruption)'].plot(kind='box')
plt.figure()
df['Trust (Government Corruption)'].plot(kind='hist', bins=50)

## 4.4. Dystopia Residual

In [ ]:
lowd, highd = detect_outlier(df["Dystopia Residual"])

In [ ]:
lowd

In [ ]:
highd

In [ ]:
df[df["Dystopia Residual"] > lowd]

In [ ]:
df[df["Dystopia Residual"] < highd]

In [ ]:
df[(df["Dystopia Residual"] > lowd) & (df["Dystopia Residual"] < highd)]

In [ ]:
df = df[(df["Dystopia Residual"] > lowd) & (df["Dystopia Residual"] < highd)]
df["Dystopia Residual"].plot(kind="box")
plt.figure()
df["Dystopia Residual"].plot(kind="hist", bins=50)

* Aykırı değer analizi yapıldıktan sonra df'imizdeki gözlemlerimizi ve kolonlarımızı inceleyelim.Görselleştirmeye hazır hale getirelim

* Tekrardan aykırı değer ler gittikten sonraki boxplotlarımız görüntüleyelim.Verilerl arasında nasıl bir değişiklik olmuş iyice inceleyelim

In [ ]:
Image(filename="../input/imagess/Before.png")

In [ ]:
fig, [[ax1, ax2, ax3, ax4, ax5], [ax6 ,ax7, ax8, ax9, ax10]] = plt.subplots(nrows=2,
                                      ncols=5, figsize=(14, 20))

ax1.boxplot(df["Happiness Rank"])
ax1.set_title("Happiness Rank")
ax2.boxplot(df["Happiness Score"])
ax2.set_title("Happiness Score")
ax3.boxplot(df["Standard Error"])
ax3.set_title("Standard Error")
ax4.boxplot(df["Economy (GDP per Capita)"])
ax4.set_title("Economy (GDP per Capita)")
ax5.boxplot(df["Family"])
ax5.set_title("Family")
ax6.boxplot(df["Health (Life Expectancy)"])
ax6.set_title("Health (Life Expectancy)")
ax7.boxplot(df["Freedom"])
ax7.set_title("Freedom")
ax8.boxplot(df["Trust (Government Corruption)"])
ax8.set_title("Trust (Government Corruption)")
ax9.boxplot(df["Generosity"])
ax9.set_title("Generosity")
ax10.boxplot(df["Dystopia Residual"])
ax10.set_title("Dystopia Residual")

In [ ]:
df.shape

In [ ]:
df.head()

* df içerisindeki korelasyonu incelemek adına bir heatmap görüntüleyelim

In [ ]:
f, ax = plt.subplots(figsize= [20, 15])
sns.heatmap(df.corr(), annot=True, ax=ax, cmap="BuPu")

# 5. Veri Görselleştirme

* Özellikler arasındaki ilşkiyi yakından görebilelim diye bir pairplot neyin ne olduğu açısından iyi olabilir

In [ ]:
palette =sns.color_palette("husl", 8)
palette

In [ ]:
sns.pairplot(df, palette=palette[7])

In [ ]:
df.head()

## 5.1. World Map

### 5.1.1 Happiness Score - Country

* Ülkelere göre mutluluk oranına bağlı dünya haritası çıkaralım

In [ ]:
map_happy = dict(type='choropleth', 
           locations=df['Country'],
           locationmode='country names',
           z=df['Happiness Score'], 
           text=df['Country'],
           colorbar={'title':'Happiness Score'},
           colorscale='BuPu')
layout = dict(title='Happiness Score Across The World by Country', 
              geo=dict(showframe=False, projection=
                         {'type': 'equirectangular'}))
choromap3 = go.Figure(data=[map_happy], layout=layout)
iplot(choromap3)

* En çok ve en az mutlu olanları analiz edelim ve görselle uyumunu inceleyelim

In [ ]:
df.groupby(df["Country"]).sum()["Happiness Score"].max()

In [ ]:
df[df["Happiness Score"] == 7.587000000000001]

In [ ]:
df.groupby(df["Country"]).sum()["Happiness Score"].min()

In [ ]:
df[df["Happiness Score"] == 2.839]

* Mesela bu bilgiye göre en mutlu olan Kanada,Norveç,İzlanda ve amaerika iken en az mutlu olanlar Afrika toprakalrında yaşayan kimseler olarak analiz ediliyor.Bu durumda bizi aslında ekonomik açıdan mutluluk kayanğını sorgulamaya itiyor

### 5.1.2 Happiness Score - Economy

In [ ]:
map_happy = dict(type='choropleth', 
           locations=df['Country'],
           locationmode='country names',
           z=df['Economy (GDP per Capita)'], 
           text=df['Country'],
           colorbar={'title': 'Economy (GDP per Capita)'},
           colorscale='Blues')
layout = dict(title='Happiness Score Across The World BY Economy', 
              geo=dict(showframe = False, projection=
                         {'type': 'equirectangular'}))
choromap3 = go.Figure(data=[map_happy], layout=layout)
iplot(choromap3)

### 5.1.3. Health (Life Expectancy) - Happiness Score

In [ ]:
df.head()

In [ ]:
map_happy = dict(type='choropleth', 
           locations=df['Country'],
           locationmode='country names',
           z=df['Health (Life Expectancy)'], 
           text=df['Country'],
           colorbar={'title': 'Health (Life Expectancy) - Happiness Score'},
           colorscale='viridis')
layout = dict(title='Happiness Score Across The World by Healthy', 
              geo=dict(showframe=False, projection=
                         {'type': 'equirectangular'}))
choromap3 = go.Figure(data=[map_happy], layout=layout)
iplot(choromap3)

### 5.1.4. Freedom - Happiness Score

In [ ]:
map_happy = dict(type = "choropleth",
                locations = df["Country"],
                locationmode = "country names",
                z = df["Freedom"],
                text = df["Country"],
                colorbar = {"title" : "Freedom"},
                colorscale="magma")
layout = dict(title="Happiness Score Across The World by Freedom",
              geo=dict(showframe=False, projection={"type": "equirectangular"}))
choromap3 = go.Figure(data=[map_happy], layout=layout)
iplot(choromap3)

### 5.1.5 Trust (Government Corruption) - Happiness Score

In [ ]:
map_happy = dict(type = "choropleth",
                locations = df["Country"],
                locationmode = "country names",
                z = df["Trust (Government Corruption)"],
                text = df["Country"],
                colorbar = {"title" : "Trust (Government Corruption)"},
                colorscale="YlGnBu")
layout = dict(title="Happiness Score Across The World by Trust That Is Given By Government",
              geo=dict(showframe=False, projection={"type": "equirectangular"}))
choromap3 = go.Figure(data=[map_happy], layout=layout)
iplot(choromap3)

## 5.2. Histograms & Scatter Plots

In [ ]:
df.head()

### 5.2.1. Standard Error - Happiness Score

In [ ]:
sns.jointplot(x=df["Standard Error"], y=df["Happiness Score"],
              data=df,
              joint_kws={"s": 5, "color": "purple"},
              marginal_kws={"color": "darkblue"})

### 5.2.2. Economy - Happiness Score

In [ ]:
sns.jointplot(x=df["Economy (GDP per Capita)"], y=df["Happiness Score"],
              data=df, joint_kws={"s": 5, "color": "green"},
              marginal_kws={"color": "green"})

### 5.2.3. Generosity - Happiness Score

In [ ]:
sns.jointplot(x=df["Generosity"], y=df["Happiness Score"],
              data=df, joint_kws={"s": 5, "color": "orange"},
              marginal_kws={"color": "orange"})

### 5.2.4. Dystopia Residual - Happiness Score

In [ ]:
sns.jointplot(x=df["Dystopia Residual"], y=df["Happiness Score"],
              data=df, joint_kws={"s": 5, "color": "#ff6162"},
              marginal_kws={"color": "#ff6162"})

## 5.3. Scatter Plots

* Bütün özelliklere ait mutluluk skorunu etkileyecek grafik için violin grafiklerini toplu şekilde çizdirelim

In [ ]:
df.head()

In [ ]:
qualitative_colors = sns.color_palette("Set2", 10)
sns.palplot(qualitative_colors)
type(qualitative_colors[0])

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(18, 18))
plt.suptitle("Hedef ile Özelliklerin Scatter Grafikleri",
             fontsize=25)
sns.scatterplot(x="Happiness Score", y="Happiness Rank", data=df, ax=ax[0, 0],
               color=qualitative_colors[0])
sns.scatterplot(x="Happiness Score", y="Economy (GDP per Capita)", data=df, ax=ax[0, 1],
               color=qualitative_colors[1])
sns.scatterplot(x="Happiness Score", y="Family", data=df, ax=ax[0, 2],
               color=qualitative_colors[2])
sns.scatterplot(x="Happiness Score", y="Health (Life Expectancy)", data=df, ax=ax[1, 0],
               color=qualitative_colors[3])
sns.scatterplot(x="Happiness Score", y="Freedom", data=df, ax=ax[1, 1],
               color=qualitative_colors[4])
sns.scatterplot(x="Happiness Score", y="Trust (Government Corruption)", data=df, ax=ax[1, 2],
               color=qualitative_colors[5])
sns.scatterplot(x="Happiness Score", y="Generosity", data=df, ax=ax[2, 0],
               color=qualitative_colors[6])
sns.scatterplot(x="Happiness Score", y="Dystopia Residual", data=df, ax=ax[2, 1],
               color=qualitative_colors[7])
sns.scatterplot(x="Happiness Score", y="Standard Error", data=df, ax=ax[2, 2],
               color=qualitative_colors[8])

## 6. Özellikler Arası Analiz

### 6.1.1. En Çok Mutlu Ülkeler

In [ ]:
happy2015_top10 = df.groupby('Country')['Happiness Score'].max(
).sort_values(ascending=False).head(10).reset_index()
happy2015_top10

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(y=happy2015_top10['Country'],
            x=happy2015_top10['Happiness Score'],
            palette='rocket', orient='h')
plt.title('Top 10 Happiest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.2. En Az Mutlu Ülkeler

In [ ]:
happy2015_bottom10 = df.groupby("Country")["Happiness Score"].max(
).sort_values(ascending=True).head(10).reset_index()
happy2015_bottom10

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(y=happy2015_bottom10['Country'],
            x=happy2015_bottom10['Happiness Score'],
            palette="mako", orient='h')
plt.title('Top 10 Saddest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.3. En Yüksek GDP Oranına Sahip Ülkeler 

In [ ]:
gdp2015_top10 = df.groupby("Country")["Economy (GDP per Capita)"].max(
).sort_values(ascending=False).head(10).reset_index()
gdp2015_top10

In [ ]:
plt.figure(figsize=[10, 7])
sns.barplot(x=gdp2015_top10["Economy (GDP per Capita)"],
            y=gdp2015_top10["Country"],
            palette="rocket", orient='h')
plt.title('Top 10 GDP Highest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.4. En Düşük GDP Oranına Sahip Ülkeler

In [ ]:
gdp2015_bottom10 = df.groupby("Country")["Economy (GDP per Capita)"].max(
).sort_values(ascending=True).head(10).reset_index()
gdp2015_bottom10

In [ ]:
plt.figure(figsize=[10, 7])
sns.barplot(x=gdp2015_bottom10["Economy (GDP per Capita)"],
            y=gdp2015_bottom10["Country"],
            palette="mako", orient='h')
plt.title('Top 10 GDP Lowest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.5.  En Yüksek Sağlık Beklentisine Sahip Ülkeler

In [ ]:
health2015_top10 = df.groupby("Country")["Health (Life Expectancy)"].max(
).sort_values(ascending=False).head(10).reset_index()
health2015_top10

In [ ]:
plt.figure(figsize=[10, 7])
sns.barplot(x=health2015_top10["Health (Life Expectancy)"],
            y=health2015_top10["Country"],
            palette="rocket", orient='h')
plt.title('Top 10 Health Expectancy Highest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.6 En Düşük Sağlık Beklentisine Sahip Ülkeler

In [ ]:
health2015_bottom10 = df.groupby("Country")["Health (Life Expectancy)"].max(
).sort_values(ascending=True).head(10).reset_index()
health2015_bottom10

In [ ]:
plt.figure(figsize=[10, 7])
sns.barplot(x=health2015_bottom10["Health (Life Expectancy)"],
            y=health2015_bottom10["Country"],
            palette="mako", orient='h')
plt.title('Top 10 Health Expectancy Lowest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.7. Trust Oranı En Yüksek 10 Ülke

In [ ]:
trust2015_top10 = df.groupby("Country")["Trust (Government Corruption)"].max(
).sort_values(ascending=False).head(10).reset_index()
trust2015_top10

In [ ]:
plt.figure(figsize=[10, 7])
sns.barplot(x=trust2015_top10["Trust (Government Corruption)"],
            y=trust2015_top10["Country"],
            palette="rocket", orient='h')
plt.title('Top 10 Trust Highest Country', fontsize=15)
plt.xlabel('Happiness Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 8)

### 6.1.8 Trust Oranı En Düşük 10 Ülke

In [ ]:
trust2015_bottom10 = df.groupby("Country")[
    "Trust (Government Corruption)"].max(
).sort_values(ascending=True).head(10).reset_index()
trust2015_bottom10

* Boş değerler olduğu için 0.0... gibi bunları visualize ederken çıktılar doğrudan görünmeyecek,bu yüzden görselleştirmeye ihtiyacımız yok

# 7. Kümeleme - KMeans

* Kümeleme bir veri kümesinde benzer özellik gösteren verilerin gruplara ayrılmasına denir.Her veri ile rastgele belirlenen merkez noktalarını daha iyi seçebilmek için başlangıç noktası ile diğer veriler arasındaki mesafe hesaplanır.Bu mesafenin karesi alınarak,belli hesaplamalar yaparak yeni başlangıç noktaları seçilir.

In [ ]:
vals = df.iloc[:, 2:-1].values
vals

In [ ]:
km = KMeans(n_clusters=3, init="k-means++", random_state=2)
km.fit(vals)

In [ ]:
predict = km.predict(vals)

In [ ]:
km.cluster_centers_

In [ ]:
plt.scatter(vals[predict==0, 0], vals[predict==0, 1], s=50, color='purple')
plt.scatter(vals[predict==1, 0], vals[predict==1, 1], s=50, color='blue')
plt.scatter(vals[predict==2, 0], vals[predict==2, 1], s=50, color='pink')
plt.title('K-Means Happiness Dataset')
plt.show()

# 8. Sonuç

KMeans algoritması sonucunda mutluluk durumlarına bağlı olarak 3 farklı gruba ayırmasını istedim ve belirgin bir şekilde kümelendiğini görebiliyorum.KMeans bu algoritma için en iyi algoritmalardan biri olabilir